# 북한 기사 상위 키워드 빈도 비교
작성자: 김지민, 현지영

이 노트북은 북한 기사 데이터를 바탕으로 도발 1달 전 기사에서 많이 등장한 단어들을 분석하고,  
- 기사 파일별로 자주 등장한 단어 100개를 정리한 데이터(`*_북한기사_전체.csv`)를 수집  
- 이를 통합하여 전체적으로 많이 쓰인 단어 상위 100개를 추출  
- 향후 북한 도발 전의 언론 특성 및 키워드 경향 분석에 활용 가능  

## 주요 내용
### 1. 불용어 지정
- 분석에서 제외할 불필요한 단어(예: 조사, 접속사, 뉴스 문구 등) 리스트 정의  
- 불용어 리스트는 `stopwords` 딕셔너리로 구성되어 형태소 분석 후 제거됨  

### 2. 도발 1달 전 북한 기사(1~15번) 각각 자주 나온 단어 100개 뽑기  
- 각 기사 CSV 파일에서 `Title`, `Content`를 추출 후 형태소 분석  
- 명사, 형용사, 동사 중 길이 2 이상인 단어만 필터링  
- 파일별로 상위 100개 단어를 빈도와 함께 컬럼 형식으로 저장  
- 결과 파일 예시: `북한기사_15사건_top100.csv`  

### 3. 도발 1달 전 북한 기사(1~14번)를 통합하여 자주 나온 단어 100개 뽑기  
- 앞선 파일들 중 번호가 1~14인 기사만 모아 전체 빈도 분석  
- 통합 빈도 기반으로 상위 100개 키워드를 선정해 `북한_도발전_통합_top100.csv`로 저장  
- 향후 워드클라우드, 키워드 네트워크 분석 등으로 확장 가능

---

In [1]:
import pandas as pd
import glob
import os
from collections import Counter
from konlpy.tag import Okt

-------

## 1. 불용어 지정

In [2]:
stopwords = {
    '하였다', '하여', '하는', '하고있다', '다음', '였다', '되였다', '하기', '있다', '많은',
    '할수', '하면서', '대해', '한다', '하시였다', '따라', '하며', '누구', '때문', '위해',
    '어떤', '무슨', '위대한', '있는', '지금', '아니라', '그것', '입니다', '모든', '없다',
    '하신', '대하', '되어', '되여', '아니라', '바로', '지난해', '하면', '자기', '사실', '지난', 
    '즈음', '되는', '의해', '되고', '않는', '합니다', '한다고', '같은', '있었다', '여러', '해야',
    '없는', '더욱', '대한', '이어', '가장', '관련', '오늘', '이런', '하게', '비롯',
    '다른', '높이', '하고', '계속', '이니', '하고있는', '자주', '우리', '여기', '최근', '이번', '있습니다',
    '전체', '되었습니다', '했습니다', '또한', '깊은','되고있다', '로부터', '하지', '올해', '보고', '어떻게',
    '된다', '이러한', '있게', '그이', '않고', '하였습니다', '그대로', '했다', '무단', '배포', '금지', '저작권',
    '공유', '됐다', '통해', '기사', '댓글', '스크랩', '국방일보', '이었다', '밝혔다', '동지', '우리', '페이스북', 
    '트위터', '밴드', '카카오톡', 'url복사', '기자', '글=', '사진', '북한', '남한', '러시아', '미국', '중국', '한국', '남조선'
}

-----

## 2. 도발 1달 전 북한 기사(1~15번) 각각 자주 나온 단어 100개 뽑기
### 북한기사 상위 단어 추출 및 파일별 분리 저장 함수 (`save_top_words_split_columns`)
- `북한기사` 폴더 내 다수의 CSV 파일(`*_북한기사_전체.csv`)을 처리  
- 각 파일의 제목(`Title`)과 본문(`Content`)에서 텍스트 추출  
- Okt 형태소 분석기를 사용하여 **명사, 동사, 형용사**만 필터링  
- 불용어(stopwords) 제거 및 **2자 이상 단어**만 유지  
- 파일별로 상위 `top_n`개 단어를 추출하여  
  - `파일명_단어`, `파일명_빈도` 형태의 열(column)로 저장  
  - 예: `1_북한기사_전체_단어`, `1_북한기사_전체_빈도`
- 열 단위로 분리된 결과를 하나의 DataFrame으로 구성  
- 최종 결과는 `북한기사_15사건_top100.csv`로 저장됨 

In [3]:
def save_top_words_split_columns(file_list, save_path, top_n=100):
    okt = Okt()
    result_dict = {}

    for file in file_list:
        df = pd.read_csv(file)
        texts = df['Title'].dropna().astype(str).tolist() + df['Content'].dropna().astype(str).tolist()

        tokens = []
        for line in texts:
            pos = okt.pos(line)
            tokens += [token for token, tag in pos if tag in ['Noun', 'Verb', 'Adjective']]

        filtered = [token for token in tokens if len(token) >= 2 and token not in stopwords]
        counter = Counter(filtered)
        top_items = counter.most_common(top_n) # 리스트 형식의 (단어, 빈도수) 튜플들로 반환

        filename = os.path.basename(file)
        num = filename.split('_')[0]

        word_col = f"{num}_단어"
        count_col = f"{num}_빈도수"

        result_dict[word_col] = [word for word, count in top_items]
        result_dict[count_col] = [count for word, count in top_items]

    df_result = pd.DataFrame(result_dict)
    df_result = df_result.reindex(
        sorted(df_result.columns, key=lambda x: int(x.split('_')[0])),
        axis=1
    )
    df_result.to_csv(save_path, index=False, encoding='utf-8-sig')

In [ ]:
north_files = sorted(glob.glob('data/북한기사/*_북한기사_전체.csv'))
save_top_words_split_columns(north_files, '북한기사_15사건_top100.csv', top_n=100)

In [4]:
df = pd.read_csv('북한기사_15사건_top100.csv')
df.head()

,1_단어,1_빈도수,2_단어,2_빈도수,3_단어,3_빈도수,4_단어,4_빈도수,5_단어,5_빈도수,...,11_단어,11_빈도수,12_단어,12_빈도수,13_단어,13_빈도수,14_단어,14_빈도수,15_단어,15_빈도수
0,조선,1298,평양,1165,평양,1187,조선,966,위원회,1248,...,조선,10377,인민,10489,인민,9368,중앙,2451,조선,16236
1,평양,1101,조선,1136,조선,1106,평양,908,평양,1240,...,인민,7969,최고,9462,혁명,6034,위원회,2403,평양,16041
2,중앙,742,인민,699,국가,1006,국가,714,조선,1169,...,전쟁,7766,조선,8014,건설,5093,평양,2379,중앙,10784
3,선언,563,중앙,579,혁명,853,전쟁,670,중앙,1035,...,제재,5363,령도자,7096,생산,5024,생산,2269,국가,7830
4,국가,558,위원회,552,사람,806,사람,654,국가,722,...,공화국,5236,김정은,6733,애하,4968,사람,2139,인민,7650


----

## 3. 도발 1달 전 북한 기사(1~14번)를 통합하여 자주 나온 단어 100개 뽑기
### 북한 전체 기사 중 1~14번 통합하여 기사에서 자주 등장한 단어 상위 N개를 추출하는 함수(`get_top_100_words`)
- `*_북한기사_전체.csv` 파일 중 번호가 1~14인 것만 선택
- 모든 기사 텍스트에서 명사, 동사, 형용사 추출
- 불용어를 제거하고 길이 2 이상 단어만 남김
- 가장 많이 나온 단어 100개를 반환 (DataFrame 형태)

In [5]:
def get_top_100_words(north_path, top_n=100):
    okt = Okt()

    # 1~14번 기사 파일만 가져오기
    all_files = sorted(glob.glob(f"{north_path}*_북한기사_전체.csv"))
    
    target_files = []
    for f in all_files:
        # 파일명에서 숫자 부분 추출 (예: '1_북한기사_전체.csv' → '1')
        filename = os.path.basename(f)
        number = filename.split('_')[0]
    
        # 숫자인지 확인하고 1~14 사이인지 체크
        if number.isdigit():
            number = int(number)
            if number <= 14:
                target_files.append(f)
                
    print(f"처리할 파일 수: {len(target_files)}개")

    # 모든 기사 텍스트 모으기
    all_texts = []
    for file in target_files:
        df = pd.read_csv(file)
        # 제목과 내용 가져오기
        title_texts = df['Title'].dropna().astype(str).tolist()
        content_texts = df['Content'].dropna().astype(str).tolist()
        all_texts.extend(title_texts + content_texts)

    # 형태소 분석으로 단어 추출
    tokens = []
    for sentence in all_texts:
        for word, tag in okt.pos(sentence):
            if tag in ['Noun', 'Verb', 'Adjective']:
                tokens.append(word)

    # 불용어 제거 + 2글자 이상 단어만 남기기
    clean_tokens = [token for token in tokens if len(token) >= 2 and token not in stopwords]

    # 가장 많이 나온 단어 세기
    counter = Counter(clean_tokens)
    top_words = counter.most_common(top_n)

    # DataFrame으로 반환
    df_top = pd.DataFrame(top_words, columns=['단어', '빈도수'])
    return df_top

In [ ]:
df_top100 = get_top_100_words('data/북한기사/')
df_top100.to_csv('북한_도발전_통합_top100.csv', index=False, encoding='utf-8-sig')

In [6]:
pd.read_csv('북한_도발전_통합_top100.csv')

,단어,빈도수
0,조선,54063
1,인민,52986
2,평양,43863
3,혁명,34429
4,중앙,34388
...,...,...
95,개발,7433
96,위업,7391
97,하나,7327
98,교육,7300


------